In [1]:
'''
citation:
@article{zhangCharacterlevelConvolutionalNetworks2015,
  archivePrefix = {arXiv},
  eprinttype = {arxiv},
  eprint = {1509.01626},
  primaryClass = {cs},
  title = {Character-Level { {Convolutional Networks} } for { {Text Classification} } },
  abstract = {This article offers an empirical exploration on the use of character-level convolutional networks (ConvNets) for text classification. We constructed several large-scale datasets to show that character-level convolutional networks could achieve state-of-the-art or competitive results. Comparisons are offered against traditional models such as bag of words, n-grams and their TFIDF variants, and deep learning models such as word-based ConvNets and recurrent neural networks.},
  journal = {arXiv:1509.01626 [cs]},
  author = {Zhang, Xiang and Zhao, Junbo and LeCun, Yann},
  month = sep,
  
  year = {2015},
}
'''

'\ncitation:\n@article{zhangCharacterlevelConvolutionalNetworks2015,\n  archivePrefix = {arXiv},\n  eprinttype = {arxiv},\n  eprint = {1509.01626},\n  primaryClass = {cs},\n  title = {Character-Level { {Convolutional Networks} } for { {Text Classification} } },\n  abstract = {This article offers an empirical exploration on the use of character-level convolutional networks (ConvNets) for text classification. We constructed several large-scale datasets to show that character-level convolutional networks could achieve state-of-the-art or competitive results. Comparisons are offered against traditional models such as bag of words, n-grams and their TFIDF variants, and deep learning models such as word-based ConvNets and recurrent neural networks.},\n  journal = {arXiv:1509.01626 [cs]},\n  author = {Zhang, Xiang and Zhao, Junbo and LeCun, Yann},\n  month = sep,\n  \n  year = {2015},\n}\n'

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

In [3]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])
    plt.show()

In [4]:
dataset, info = tfds.load('yelp_polarity_reviews/subwords8k', as_supervised=True, with_info=True)

In [5]:
train_examples, test_examples = dataset['train'], dataset['test']

In [6]:
(train_examples)

<PrefetchDataset shapes: ((None,), ()), types: (tf.int64, tf.int64)>

In [7]:
encoder = info.features['text'].encoder

In [8]:
print('Vocabulary size {}'.format(encoder.vocab_size))

Vocabulary size 8176


In [9]:
sample_string = 'Hello TensorFlow'
encoded_string = encoder.encode(sample_string)
print('Encoded string is {}'.format(encoded_string))

original_string = encoder.decode(encoded_string)
print('The original string: "{}"'.format(original_string))

Encoded string is [6263, 198, 1489, 1353, 559, 6598, 8039]
The original string: "Hello TensorFlow"


In [10]:
assert original_string == sample_string

In [11]:
for index in encoded_string:
  print('{} ----> {}'.format(index, encoder.decode([index])))

6263 ----> Hell
198 ----> o 
1489 ----> Te
1353 ----> ns
559 ----> or
6598 ----> Flo
8039 ----> w


In [12]:
# create batches of encoded strings
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = (train_examples
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE, padded_shapes=([None],[])))

test_dataset = (test_examples
                .padded_batch(BATCH_SIZE,  padded_shapes=([None],[])))

In [13]:
# build RNN model
"""
Embedding layer: stores one vector per word. converts sequences of word indices to sequences of vectors
Bidirectional layer: propagates the input forward and backwards and concatenates the output. Helps to learn long range dependences(?)
Dense layer: one hot encoding 
"""
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),   
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [14]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [15]:
history = model.fit(train_dataset, epochs=1,
                    validation_data=test_dataset, 
                    validation_steps=30)

8750/8750 [==============================] - 5357s 612ms/step - loss: 0.2203 - accuracy: 0.9068 - val_loss: 0.1537 - val_accuracy: 0.9417


In [16]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

    594/Unknown - 126s 213ms/step - loss: 0.1748 - accuracy: 0.9309Test Loss: 0.1747805086914638
Test Accuracy: 0.9309210777282715
